# Bike Rental Data Management

This notebook follows the management of the Citi Bike rental data.

## Import Libraries

As always, the first step is to import necessary libraries and dependencies for working on this project.

In [9]:
# Import libraries

import os
import datetime
import string
import glob

import pandas as pd
import numpy as np

import sqlalchemy

## Loading & Preparing Data

After loading the necessary libraries, the next step is to import our data and perform some checks to get an understanding of the data we'll be working with. After this, we can do some preparation to our data to make it easier to perform our transformations.

### Loading Bike Rental Data

As the data for the bike rentals is spread across multiple `.csv` files, one for each month, I will use the `glob` library to import these into pandas `DataFrame` objects. I will then use the `pd.concat()` method to concatanate these dataframes into one single dataframe.

In [10]:
# Use glob to retrieve all bike data csv files
csv_files = glob.glob('./data/JC-2016*-citibike-tripdata.csv')

# Iterate over files and open with pandas
bike_dfs = [pd.read_csv(csv_file) for csv_file in csv_files]

# Concatenate all dataframes
bike_df = pd.concat(bike_dfs)

# Reset the bike_df index after concatenation
bike_df.reset_index(drop=True, inplace=True)

# Add an 'id' column. This will be useful for later.
bike_df['id'] = bike_df.index

### Examining Bike Rental Data

Now that the bike rental data has been loaded, I will examine the `bike_df` dataframe, to get an understanding of what our data looks like. To do this, I will perform the following steps:

1. Look at the shape of the dataset using `.shape` to understand how the data is organized.
2. Look at the first five rows of `bike_df` using `.head()`.
3. Look at the last five rows of `bike_df` using `.tail()`.
4. Look at the data types across the variables in `bike_df` using `.dtypes`.
5. Use `.describe()` to get a summary of the numerical variables in `bike_df`.
6. Quantify missingness across different variables using `.isna()`.
7. Quantify duplications across different variables using `.duplicated()`.

In [24]:
# Look at the shape of the dataframe
bike_df.shape

(247584, 16)

**Observations:**
- There are 247,584 observations.
- There are 16 variables for each observation.

In [11]:
# Look at the first five rows of the dataframe
bike_df.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2,0
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1,3
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,4


**Observations:**
- The dataset contains both categorical and numerical variables.
- There appears to be some missingness in the `Birth Year` column.

In [12]:
# Look at the last five rows of the dataframe
bike_df.tail()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
247579,557,2016-12-31 23:10:16,2016-12-31 23:19:33,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,24465,Subscriber,1981.0,2,247579
247580,2749,2016-12-31 23:29:39,2017-01-01 00:15:29,3183,Exchange Place,40.716247,-74.033459,3183,Exchange Place,40.716247,-74.033459,24389,Customer,NaN,0,247580
247581,173,2016-12-31 23:44:37,2016-12-31 23:47:31,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,24641,Subscriber,1978.0,1,247581
247582,2424,2016-12-31 23:44:50,2017-01-01 00:25:14,3214,Essex Light Rail,40.712774,-74.036486,3214,Essex Light Rail,40.712774,-74.036486,26219,Subscriber,1960.0,2,247582
247583,2419,2016-12-31 23:44:50,2017-01-01 00:25:10,3214,Essex Light Rail,40.712774,-74.036486,3214,Essex Light Rail,40.712774,-74.036486,24471,Subscriber,1956.0,1,247583


**Observations:**
- More missingness in the `Birth Year` column.
- There are at least two different values for `User Type`:
    - Subscriber
    - Customer

In [13]:
# Look at the dataframe datatypes
bike_df.dtypes

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
id                           int64
dtype: object

**Observations:**
- Pandas appears to have interpreted most data types correctly, except for `Start Time` and `Stop Time`. These will need to be cast as datetime variables to allow for more effective transformation.
- There is a variety of string, integer and float data types.

In [14]:
# Look at summary of numerical variables
bike_df.describe()

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender,id
count,2.475840e+05,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000,247584.000000,247584.000000
mean,8.856305e+02,3207.065206,40.723121,-74.046438,3203.572553,40.722594,-74.045855,24935.260481,1979.335276,1.123534,123791.500000
std,3.593798e+04,26.955103,0.008199,0.011211,61.579494,0.007958,0.011283,748.469712,9.596809,0.518687,71471.488861
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,40.692216,-74.096937,14552.000000,1900.000000,0.000000,0.000000
25%,2.480000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24491.000000,1974.000000,1.000000,61895.750000
50%,3.900000e+02,3201.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043117,24609.000000,1981.000000,1.000000,123791.500000
75%,6.660000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24719.000000,1986.000000,1.000000,185687.250000
max,1.632981e+07,3426.000000,40.752559,-74.032108,3426.000000,40.801343,-73.957390,27274.000000,2000.000000,2.000000,247583.000000


**Observations:**
- Maximum `Trip Duration` is five orders of magnitude longer than that of the 75th percentile.
- The minimum value for `Birth Year` is 1900, which would correspond with an age of 116 at the time the data was recorded.

In [15]:
# Quantify missing values
bike_df.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
id                             0
dtype: int64

**Observations:**
- `Birth Year` and `User Type` are the only two variables with missingness.
- `Birth Year` by far has the greatest number of missing values.

In [16]:
# Quantify duplicate values
bike_df.duplicated().sum()

np.int64(0)

**Observations:**
- There do not appear to be any duplicated observations across the dataset.

### Loading Weather Data

Now, I will load in the data gathered by Newark Airport for the weather over the same time period as the bike rental data. As this is a single `.csv` file, I can simply use the `.read_csv()` method in pandas.

In [17]:
# Load weather data
weather_df = pd.read_csv('./data/newark_airport_2016.csv')

### Examining Weather Data

Now I will examine the `weather_df` using the same steps as I did for `bike_df`.

In [25]:
# Look at the shape of the dataframe
weather_df.shape

(366, 16)

**Observations:**
- There are 366 observations (one for each day of the year).
- There are 16 variables for each observation.

In [18]:
# Look at the first five rows of the dataframe
weather_df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


**Observations:**
- The variable names are not very descriptive.
- There appear to be a significant number of missing values for both `TSUN` AND `PGTM`.

In [19]:
# Look at the last five rows of the dataframe
weather_df.tail()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
361,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-27,13.65,NaN,0.01,0.0,0.0,53,62,40,NaN,270,270.0,29.1,38.0
362,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-28,8.28,NaN,0.00,0.0,0.0,41,43,31,NaN,330,330.0,19.9,25.1
363,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-29,8.05,NaN,0.36,0.0,0.0,38,45,31,NaN,170,150.0,18.1,25.1
364,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-30,14.99,NaN,0.00,0.0,0.0,37,42,32,NaN,270,270.0,25.9,33.1
365,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-31,12.30,NaN,0.00,0.0,0.0,35,44,29,NaN,200,220.0,21.9,28.0


**Observations:**
- Further missingness across `TSUN` AND `PGTM` variables.

In [20]:
# Look at the dataframe datatypes
weather_df.dtypes

STATION     object
NAME        object
DATE        object
AWND       float64
PGTM       float64
PRCP       float64
SNOW       float64
SNWD       float64
TAVG         int64
TMAX         int64
TMIN         int64
TSUN       float64
WDF2         int64
WDF5       float64
WSF2       float64
WSF5       float64
dtype: object

**Observations:**
- Again, the `DATE` variable would be best transformed into a `date` object.

In [21]:
# Look at summary of numerical variables
weather_df.describe()

,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
count,366.000000,0.0,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,0.0,366.000000,364.000000,366.000000,364.000000
mean,9.429973,NaN,0.104945,0.098087,0.342623,57.196721,65.991803,48.459016,NaN,217.841530,228.269231,20.484426,26.801648
std,3.748174,NaN,0.307496,1.276498,2.078510,17.466981,18.606301,17.135790,NaN,102.548282,97.415777,6.848390,8.882610
min,2.460000,NaN,0.000000,0.000000,0.000000,8.000000,18.000000,0.000000,NaN,10.000000,10.000000,6.900000,10.100000
25%,6.765000,NaN,0.000000,0.000000,0.000000,43.000000,51.250000,35.000000,NaN,150.000000,150.000000,15.000000,19.900000
50%,8.720000,NaN,0.000000,0.000000,0.000000,56.000000,66.000000,47.000000,NaN,240.000000,260.000000,19.900000,25.100000
75%,11.410000,NaN,0.030000,0.000000,0.000000,74.000000,83.000000,64.000000,NaN,300.000000,300.000000,23.900000,31.100000
max,22.820000,NaN,2.790000,24.000000,20.100000,89.000000,99.000000,80.000000,NaN,360.000000,360.000000,48.100000,66.000000


In [22]:
# Quantify missing values
weather_df.isna().sum()

STATION      0
NAME         0
DATE         0
AWND         0
PGTM       366
PRCP         0
SNOW         0
SNWD         0
TAVG         0
TMAX         0
TMIN         0
TSUN       366
WDF2         0
WDF5         2
WSF2         0
WSF5         2
dtype: int64

**Observations:**
- `PGTM` and `TSUN` have no present values for the entire dataset.
- `WDF5` has two missing values.
- `WSF5` has two missing values.

In [23]:
# Quantify duplicate values
weather_df.duplicated().sum()

np.int64(0)

**Observations:**
- Just like with the bike rental data, there are no missing values for the weather dataset.